In [1]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import warnings,sys, json, itertools, random, heapq
warnings.filterwarnings('ignore')
from tqdm import tqdm
from pprint import pprint
import shapely.ops
from shapely.geometry import shape, Point
import geopandas as gpd
from concurrent.futures import ThreadPoolExecutor, as_completed
import networkx as nx

In [5]:
import random
from collections import deque

class Warehouse:
    def __init__(self, seed):
        # random.seed(seed)
        self.N = random.randint(5, 10)
        self.M = random.randint(5, 10)
        self.P = random.randint(2, 6)
        self.O = random.randint(1, 10)
        self.start = (0, 0)
        self.packages = []
        self.dropoffs = []
        self.obstacles = set()
        
        occupied = {self.start}
        
        # Generate packages and dropoffs
        for _ in range(self.P):
            while True:
                p = (random.randint(0, self.N-1), random.randint(0, self.M-1))
                if p not in occupied:
                    break
            occupied.add(p)
            
            while True:
                d = (random.randint(0, self.N-1), random.randint(0, self.M-1))
                if d not in occupied:
                    break
            occupied.add(d)
            
            self.packages.append(p)
            self.dropoffs.append(d)
        
        # Generate obstacles
        for _ in range(self.O):
            while True:
                o = (random.randint(0, self.N-1), random.randint(0, self.M-1))
                if o not in occupied:
                    self.obstacles.add(o)
                    occupied.add(o)
                    break

    def display_grid(self):
        grid = [['.' for _ in range(self.M)] for _ in range(self.N)]
        grid[self.start[0]][self.start[1]] = 'S'
        
        for x, y in self.packages:
            grid[x][y] = 'P'
        for x, y in self.dropoffs:
            grid[x][y] = 'D'
        for x, y in self.obstacles:
            grid[x][y] = 'O'
        
        print("Initial Warehouse Configuration:")
        for row in grid:
            print(' '.join(row))
        print()

class Agent:
    def __init__(self, warehouse):
        self.warehouse = warehouse
        self.current_pos = warehouse.start
        self.total_cost = 0
        self.total_reward = 0
        self.penalties = 0
        self.path_taken = [warehouse.start]

    def get_neighbors(self, pos):
        x, y = pos
        neighbors = []
        for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < self.warehouse.N and 0 <= ny < self.warehouse.M:
                neighbors.append((nx, ny))
        return neighbors

    def bfs(self, start, goal):
        queue = deque([[start]])
        visited = set([start])
        
        while queue:
            path = queue.popleft()
            current = path[-1]
            
            if current == goal:
                return path
            
            for neighbor in self.get_neighbors(current):
                if neighbor not in visited and neighbor not in self.warehouse.obstacles:
                    visited.add(neighbor)
                    new_path = list(path)
                    new_path.append(neighbor)
                    queue.append(new_path)
        return None

    def deliver_packages(self):
        for i in range(len(self.warehouse.packages)):
            # Move to package
            pkg_path = self.bfs(self.current_pos, self.warehouse.packages[i])
            if not pkg_path:
                print(f"Package {i+1} unreachable!")
                continue
                
            self.path_taken += pkg_path[1:]
            self.total_cost += len(pkg_path) - 1
            self.current_pos = self.warehouse.packages[i]
            
            # Move to dropoff
            drop_path = self.bfs(self.current_pos, self.warehouse.dropoffs[i])
            if not drop_path:
                print(f"Dropoff {i+1} unreachable!")
                continue
                
            self.path_taken += drop_path[1:]
            self.total_cost += len(drop_path) - 1
            self.current_pos = self.warehouse.dropoffs[i]
            self.total_reward += 10
            
        # Calculate final score
        final_score = self.total_reward - self.total_cost - (self.penalties * 5)
        
        return {
            'path': self.path_taken,
            'total_cost': self.total_cost,
            'total_reward': self.total_reward,
            'penalties': self.penalties,
            'final_score': final_score
        }

# Example usage
seed = 42
warehouse = Warehouse(seed)
warehouse.display_grid()

agent = Agent(warehouse)
results = agent.deliver_packages()

print(f"Chosen Path: {results['path']}")
print(f"Total Cost: {results['total_cost']}")
print(f"Total Reward: {results['total_reward']}")
print(f"Penalties: {results['penalties']}")
print(f"Final Score: {results['final_score']}")


Initial Warehouse Configuration:
S . P O . P D D
. O . D . . . .
. P . . P . . .
. D . . O . . .
. P . O P . . .
. O . . . D D .

Chosen Path: [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (3, 2), (4, 2), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (4, 6), (3, 6), (2, 6), (2, 5), (2, 4), (1, 4), (1, 3), (2, 3), (2, 2), (2, 1), (2, 2), (1, 2), (1, 3), (1, 4), (0, 4), (0, 5), (0, 6), (0, 5), (0, 6), (0, 7), (1, 7), (2, 7), (2, 6), (2, 5), (2, 4), (2, 3), (3, 3), (3, 2), (4, 2), (4, 1), (3, 1), (4, 1), (4, 2), (5, 2), (5, 3), (5, 4), (4, 4), (5, 4), (5, 5)]
Total Cost: 50
Total Reward: 60
Penalties: 0
Final Score: 10
